In [66]:
import pandas as pd
import numpy as np
import gspread
import oauth2client.client, oauth2client.file, oauth2client.tools
import json

In [67]:
gc = gspread.service_account(filename='creds/lifts-service.json')


In [68]:
sheet = gc.open_by_key('1Y6cVf0_oyY4xDz_Fq_NXcMjxpsmGClZCt8hDw3RIqkM')
all_worksheets = sheet.worksheets()
all_worksheets

[<Worksheet '04/01/2022' id:0>,
 <Worksheet '29/01/22' id:307277220>,
 <Worksheet '13/02/22' id:167887834>,
 <Worksheet '28/02/22' id:1331411127>,
 <Worksheet '15/03/22 DELOAD' id:585663713>,
 <Worksheet '28/03/22' id:661285306>,
 <Worksheet '11/04/22' id:1533557140>,
 <Worksheet '04/05/22' id:789390216>,
 <Worksheet '17/05/2022' id:1263314912>,
 <Worksheet '04/06/2022' id:1037833865>,
 <Worksheet '19/06/2022' id:1070839500>,
 <Worksheet '02/07/2022' id:1830989469>,
 <Worksheet '17/07/2022' id:1304829489>,
 <Worksheet '30/07/2022' id:286974438>]

In [88]:
exercises = set()
for ws in all_worksheets:
    exe = ws.col_values(1)
    for e in exe:

        exercises.add(e)

exercises = list(exercises)
exercises_only = []

[exercises_only.append(ex) for ex in list(filter(None, exercises)) if ex[0].isdigit()]

exercises_only.sort()
exercises_only

['2x10-12 Leg Curls',
 '2x10-12 Leg Extensions',
 '2x10-15 Chest Flys - DB',
 '2x10-15 Chest Flys - machine',
 '2x12-15 Bicep Isolation - DB Hammer',
 '2x12-15 Bicep Isolation - DB Waiter curl',
 '2x12-15 Dips',
 '2x12-15 Tricep Isolation - Tricep Pushdowns',
 '2x12-15 Tricep Isolation - ez bar skull crusher',
 '3x10-12 Bicep Isolation - DB curl (incline bench)',
 '3x10-12 Bicep Isolation - ez Bar Curl',
 '3x10-12 Tricep Isolation - Tricep Pushdown',
 '3x10-12 Tricep Isolation - ez bar skull crushers',
 '3x10-15 Lateral Raises - Cable',
 '3x10-15 Lateral Raises - DB',
 '3x10-15 Rear Delts - Face Pulls',
 '3x12-15 Bicep Isolation - DB Waiter curl',
 '3x5-8 Bench Press',
 '3x5-8 Rows - Barbell',
 '3x5-8 Rows - Pendlay',
 '3x6-8 BB Hip Thrusts',
 '3x6-8 Bench Press (close grip)',
 '3x6-8 Incline Press - Barbell',
 '3x6-8 Incline Press - Smith',
 '3x6-8 Leg Press',
 '3x6-8 OHP',
 '3x6-8 Pull Ups',
 '3x6-8 RDL - Barbell',
 '3x6-8 RDL - DB',
 '3x6-8 Squats - High Bar',
 '3x6-8 Squats - Low B

In [86]:
import time

test = ['3x5-8 Bench Press', '2x10-12 Leg Curls']

li = []

for ex in exercises_only: 

    record = {
        "exercise": ex,
        "progress": []
    }
    print(ex)
    time.sleep(15)
    for ws in all_worksheets:
        result_row_cell = ws.findall(ex)
        for cell in result_row_cell:
            row = ws.row_values(cell.row)
            weight = row[1]
            reps = list()
            for i in range(2, 2+int(row[0][0])):
                try:
                    reps.append(row[i])
                except(IndexError):
                    print('No value for rep_number {rep_number} for {exercise} on {worksheet}'.format(rep_number=str(i-1), exercise=ex, worksheet=ws.title))
                    pass

            record["progress"].append({
                "weight": weight,
                "reps": reps,
                "date": ws.title
            })

    li.append(record)

li



            

2


IndexError: string index out of range

In [70]:
exercises_only = []

[exercises_only.append(ex) for ex in list(filter(None, exercises)) if ex[0].isdigit()]

exercises_only.sort()

no_sets = list()

for ex in exercises_only:
    split = ex.split(" ")
    joined = ' '.join(split[1:])
    no_sets.append(joined)


unique = set(no_sets)
unique = list(unique)
unique.sort()
unique

['Arnie Press',
 'BB Hip Thrusts',
 'Bench Press',
 'Bench Press (close grip)',
 'Bench Press (decline)',
 'Bench Press - DB',
 'Bicep Isolation - DB Hammer',
 'Bicep Isolation - DB Waiter curl',
 'Bicep Isolation - DB curl (incline bench)',
 'Bicep Isolation - ez Bar Curl',
 'Calf Raises - Seated',
 'Calf Raises - Standing',
 'Chest Flys - DB',
 'Chest Flys - machine',
 'DB Shrugs',
 'Dips',
 'Incline Press - Barbell',
 'Incline Press - DB',
 'Incline Press - Smith',
 'Lateral Raises - Cable',
 'Lateral Raises - DB',
 'Leg Curls',
 'Leg Extensions',
 'Leg Press',
 'Lunges',
 'OHP',
 'Pull Downs',
 'Pull Ups',
 'RDL - Barbell',
 'RDL - DB',
 'Rear Delts - Face Pulls',
 'Rows - Barbell',
 'Rows - Cable',
 'Rows - DB',
 'Rows - Pendlay',
 'Split Squat',
 'Split Squat / Lunges',
 'Squats - High Bar',
 'Squats - Low Bar',
 'Tricep Isolation - Tricep Pushdown',
 'Tricep Isolation - Tricep Pushdowns',
 'Tricep Isolation - ez bar skull crusher',
 'Tricep Isolation - ez bar skull crushers',
 '